In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from train import train_dual_encoder

train_dual_encoder(
    train_csv="/content/drive/MyDrive/CapstoneProject/Datasets/Flickr8k_text/flickr8k_train.csv",
    image_dir="/content/drive/MyDrive/CapstoneProject/Datasets/Flickr8k_images/",
    vocab_path="/content/vocab.pkl",
    output_path="/content/dual_encoder.pth"
)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 184MB/s]


Epoch 1/5, Loss: 0.0210
Epoch 2/5, Loss: 0.0197
Epoch 3/5, Loss: 0.0192
Epoch 4/5, Loss: 0.0188
Epoch 5/5, Loss: 0.0185
Model saved to /content/dual_encoder.pth


In [ ]:
!pip install datasets torchvision nltk

In [1]:
from train import train_dual_encoder
from datasets import load_dataset

In [2]:
ds = load_dataset("jxie/flickr8k")
train_dual_encoder(
    dataset=ds["train"],
    vocab_path="vocab.pkl",
    output_path="dual_encoder.pth",
    mode="hf",
    caption_field="caption_0"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1/5, Loss: 0.0221
Epoch 2/5, Loss: 0.0210
Epoch 3/5, Loss: 0.0205
Epoch 4/5, Loss: 0.0201
Epoch 5/5, Loss: 0.0199
Model saved to dual_encoder.pth


In [ ]:
ds = load_dataset("jxie/flickr8k")
print(ds["train"].features)
# Or for a quick peek at the first row's keys:
# print(ds["train"][0].keys())

{'image': Image(mode=None, decode=True), 'caption_0': Value('string'), 'caption_1': Value('string'), 'caption_2': Value('string'), 'caption_3': Value('string'), 'caption_4': Value('string')}


In [ ]:
from datasets import load_dataset
ds = load_dataset("jxie/flickr8k")
print(ds["train"].column_names)
print(ds["train"][0])

['image', 'caption_0', 'caption_1', 'caption_2', 'caption_3', 'caption_4']
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x399 at 0x7FAA797484A0>, 'caption_0': 'A black dog is running after a white dog in the snow .', 'caption_1': 'Black dog chasing brown dog through snow', 'caption_2': 'Two dogs chase each other across the snowy ground .', 'caption_3': 'Two dogs play together in the snow .', 'caption_4': 'Two dogs running through a low lying body of water .'}


In [ ]:
import pickle
with open("vocab.pkl", 'rb') as f:
    vocab = pickle.load(f)
# Check the first few items
print(list(vocab.items())[:5])

[('word2int', {'<pad>': 0, '<sos>': 1, '<eos>': 2, '<unk>': 3, '10': 4, '12': 5, '19': 6, '1950s': 7, '2': 8, '25': 9, '28': 10, '281': 11, '3': 12, '30': 13, '3rd': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, 'a': 20, 'ability': 21, 'about': 22, 'above': 23, 'abseiling': 24, 'accelerates': 25, 'accordion': 26, 'acrobatic': 27, 'across': 28, 'acting': 29, 'action': 30, 'activity': 31, 'ad': 32, 'add': 33, 'adhd': 34, 'adjust': 35, 'admires': 36, 'admiring': 37, 'adult': 38, 'adults': 39, 'advertisement': 40, 'advertisements': 41, 'advertising': 42, 'aerial': 43, 'affectionately': 44, 'african': 45, 'africanamerican': 46, 'afro': 47, 'after': 48, 'afternoon': 49, 'against': 50, 'aged': 51, 'agency': 52, 'agility': 53, 'ahead': 54, 'aid': 55, 'aided': 56, 'aim': 57, 'aiming': 58, 'aims': 59, 'air': 60, 'airborne': 61, 'airplane': 62, 'airport': 63, 'alcohol': 64, 'alert': 65, 'aligator': 66, 'all': 67, 'alley': 68, 'alligator': 69, 'almost': 70, 'alone': 71, 'along': 72, 'alongside'